In [1]:
import pandas as pd

In [55]:
import pandas as pd

labels = {
    255: "0: NODATA",
    1: "1: MSZ, open spaces, low vegetation surfaces NDVI <= 0.3",
    2: "2: MSZ, open spaces, medium vegetation surfaces 0.3 < NDVI <=0.5",
    3: "3: MSZ, open spaces, high vegetation surfaces NDVI > 0.5",
    4: "4: MSZ, open spaces, water surfaces LAND < 0.5",
    5: "5: MSZ, open spaces, road surfaces",
    11: "11: MSZ, built spaces, residential, building height <= 3m",
    12: "12: MSZ, built spaces, residential, 3m < building height <= 6m",
    13: "13: MSZ, built spaces, residential, 6m < building height <= 15m",
    14: "14: MSZ, built spaces, residential, 15m < building height <= 30m",
    15: "15: MSZ, built spaces, residential, building height > 30m",
    21: "21: MSZ, built spaces, non-residential, building height <= 3m",
    22: "22: MSZ, built spaces, non-residential, 3m < building height <= 6m",
    23: "23: MSZ, built spaces, non-residential, 6m < building height <= 15m",
    24: "24: MSZ, built spaces, non-residential, 15m < building height <= 30m",
    25: "25: MSZ, built spaces, non-residential, building height > 30m",
}


class GHS:
    def __init__(
        self,
        df: pd.DataFrame,
        columns_index=["index", "values", "count"],
        dummy_index="xxyyzz",
        max_value=100,
    ):
        self.data = df[columns_index]
        self.columns = columns_index
        dummy_val = list(labels.keys())
        self.dummy_index = dummy_index
        self.dummy_df = pd.DataFrame(
            {"values": dummy_val, "index": dummy_index, "percent": 0}
        )
        self.max_value = max_value

    def gen_percent(self, data: pd.DataFrame):
        cols = self.columns
        count = cols[2]
        result = (
            data.groupby("index")
            .apply(lambda x: x.assign(percent=x[count] / x[count].sum() * 100))
            .drop(columns=[cols[0]])
            .reset_index()
        )
        return result

    def join_dummy(self, data: pd.DataFrame):
        dummy_df = self.dummy_df
        cols = self.columns
        result = pd.concat((data, dummy_df)).pivot_table(
            index = 'index',
            columns='values',
            values = 'percent', 
            fill_value = 0
        ).reset_index()
        result = result[result[cols[0]] != self.dummy_index]

        return result

    def join_percent(self, data):
        result = self.gen_percent(data)
        result = self.join_dummy(result)
        return result

    def generate_data(self):
        cols = self.columns
        drop_na_value = self.max_value
        data_with_na = self.data
        data_with_no_na = data_with_na[data_with_na[cols[1]] < drop_na_value]

        na_result = self.join_percent(data_with_na)
        no_na_result = self.join_percent(data_with_no_na)
        self.with_na_result = na_result
        self.without_na_result = no_na_result.drop(columns = [255])
        return self


In [59]:
import pandas as pd

id = [112.0, 112.0, 112.0, 112.0, 112.0, 112.0, 761.0, 761.0, 761.0, 761.0, 761.0, 761.0, 761.0, 761.0]
values = [1, 2, 3, 5, 11, 255, 1, 2, 3, 5, 11, 12, 13, 255]
count = [19309, 56, 1, 158, 15591, 80204407, 58635, 14427, 409, 68, 54375, 2790, 2246, 93460224]

# print(list(map(len, [id, values, count])))

data = {
    'index': id,
    'values': values,
    'count': count
}
result = pd.DataFrame(data)

In [61]:
result.groupby(['index', 'values']).sum().reset_index()

,index,values,count
0,112.0,1,19309
1,112.0,2,56
2,112.0,3,1
3,112.0,5,158
4,112.0,11,15591
5,112.0,255,80204407
6,761.0,1,58635
7,761.0,2,14427
8,761.0,3,409
9,761.0,5,68


In [57]:
a = GHS( df=result).generate_data()
a.with_na_result

C:\Users\Jhon\AppData\Local\Temp\ipykernel_19208\2561606981.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(percent=x[count] / x[count].sum() * 100))
C:\Users\Jhon\AppData\Local\Temp\ipykernel_19208\2561606981.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(percent=x[count] / x[count].sum() * 100))


values,index,1,2,3,4,5,11,12,13,14,15,21,22,23,24,25,255
0,112.0,0.024064,0.000070,0.000001,0.0,0.000197,0.019431,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.956237
1,761.0,0.062649,0.015415,0.000437,0.0,0.000073,0.058097,0.002981,0.0024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.857949


In [58]:
a.without_na_result

values,index,1,2,3,4,5,11,12,13,14,15,21,22,23,24,25
0,112.0,54.987897,0.159476,0.002848,0.0,0.449950,44.399829,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,761.0,44.103046,10.851448,0.307634,0.0,0.051147,40.898834,2.098533,1.689357,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
result_p = result.groupby("index").apply(
    lambda x: x.assign(percent=x["count"] / x["count"].sum() * 100)
).drop(columns=['index']).reset_index()
# result_p.reset_index()

C:\Users\Jhon\AppData\Local\Temp\ipykernel_16168\2117954098.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result_p = result.groupby("index").apply(


In [46]:
labels = {
    255: "0: NODATA",
    1: "1: MSZ, open spaces, low vegetation surfaces NDVI <= 0.3",
    2: "2: MSZ, open spaces, medium vegetation surfaces 0.3 < NDVI <=0.5",
    3: "3: MSZ, open spaces, high vegetation surfaces NDVI > 0.5",
    4: "4: MSZ, open spaces, water surfaces LAND < 0.5",
    5: "5: MSZ, open spaces, road surfaces",
    11: "11: MSZ, built spaces, residential, building height <= 3m",
    12: "12: MSZ, built spaces, residential, 3m < building height <= 6m",
    13: "13: MSZ, built spaces, residential, 6m < building height <= 15m",
    14: "14: MSZ, built spaces, residential, 15m < building height <= 30m",
    15: "15: MSZ, built spaces, residential, building height > 30m",
    21: "21: MSZ, built spaces, non-residential, building height <= 3m",
    22: "22: MSZ, built spaces, non-residential, 3m < building height <= 6m",
    23: "23: MSZ, built spaces, non-residential, 6m < building height <= 15m",
    24: "24: MSZ, built spaces, non-residential, 15m < building height <= 30m",
    25: "25: MSZ, built spaces, non-residential, building height > 30m",
}



In [65]:
dummy_val = list(labels.keys())
dummy_df = pd.DataFrame({"values" : dummy_val, 'index': "xyz", 'percent': 0})

In [67]:
pd.concat((result_p, dummy_df)).pivot_table(
    index = 'index',
    columns='values',
    values = 'percent', 
    fill_value = 0
).reset_index()

values,index,1,2,3,4,5,11,12,13,14,15,21,22,23,24,25,255
0,112.0,0.024064,0.000070,0.000001,0.0,0.000197,0.019431,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.956237
1,761.0,0.062649,0.015415,0.000437,0.0,0.000073,0.058097,0.002981,0.0024,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.857949
2,xyz,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
